In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font color="red" size="5">ch.11 데이터프레임과 시리즈</font></b>


** 주요함수 **
- columns, index : 열과 행 이름 바꾸기
- loc, iloc : 특정 데이터 조회하기
- merge() : 공통열 기준 병합
- concat() : 데이터 이어붙이기
- sort_index, sort_values : 행/열 이름 또는 값 기준으로 정렬
- mean(), median(), min(), max() : 평균, 중앙값, 최댓값, 최솟값 등
- groupby() : 기준 열로 데이터를 그룹화해서 요약 통계 내기
- melt(), pivot_table() : 와이드포맷↔롱포맷 변환
- apply(), applymap(), map() : 데이터프레임 전체나 각 요소에 함수 적용


# 1절 판다스 패키지
- 판다스(Pandas) : 데이터 분석을 위해 테이블 형식(엑셀) 데이터를 다루기 좋게 만들어주는 넘파이 기반 파이썬 라이브러리(엑셀을 파이썬으로 조작하는 기술)
    - 장점 : 파일 io, 부분 데이터 추출, 크기변경, 데이터 분할 및 병합, 정렬, 결측치 처리, 데이터 분할, 피벗과 언피벗(와이드포맷, 롱포맷)에 용이
    - [Pandas API reference] https://pandas.pydata.org/docs/reference/index.html#api
    - [Pandas docs] https://pandas.pydata.org/docs/
    
- 데이터 프레임(Data Frame) : 엑셀 시트처럼 행(Row) 과 열(Column) 로 이루어진 2차원 구조
- 시리즈(Series) : 데이터프레임의 한 열(Column) 을 의미(1차원 구조)
- 데이터프레임과 시리즈 요약 : 시리즈는 1열짜리 데이터, 데이터프레임은 여러 열짜리 표 전체다!
- DataFrame 만들기 : 딕셔너리, 리스트, csv 파일, sklearn, seaborn 등 다양한 방법으로 생성


In [5]:
import pandas as pd
pd.__version__

'1.5.3'

In [8]:
data = pd.read_csv('data/ch09_member1.csv', encoding='utf-8') #encoding='utf-8' 기본값으로 설정되어 있음.
display(data)
type(data)

,홍길동,20,kildong@hong.com,서울시 관악구
0,김길동,40,kimdong@hong.com,서울시 영등포구
1,신길동,30,sindong@hong.com,서울시 동작구


pandas.core.frame.DataFrame

In [12]:
data = pd.read_csv('data/ch09_member1.csv', 
                   header=None,
                   names=['name','age', 'email', 'address'])
data.head(2) # 최초 상위 2줄

,name,age,email,address
0,홍길동,20,kildong@hong.com,서울시 관악구
1,김길동,40,kimdong@hong.com,서울시 영등포구


In [15]:
data = pd.read_json('data/ch09_member1.json', encoding='utf-8')
data

,name,age,email,address
0,홍길동,22,a@a.com,서울
1,신길동,32,a@a.com,서울
2,김길동,42,a@a.com,서울


In [17]:
iris = pd.read_hdf('data/ch09_iris.hdf5', key='iris')
iris.head() # 상위 5개

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# 2절. 데이터 프레임 만들기
## 2.1 딕셔너리를 이용해서 데이터프레임 만들기(리스트를 이용해서도 가능함.)

In [18]:
d = {'kor':[100,90], 'mat':[95,99]} #딕셔너리를 데이터프레임으로
df = pd.DataFrame(data=d)
df

,kor,mat
0,100,95
1,90,99


In [19]:
df.to_dict() #데이터 프레임을 딕셔너리로 변환

{'kor': {0: 100, 1: 90}, 'mat': {0: 95, 1: 99}}

In [20]:
df.to_numpy()

array([[100,  95],
       [ 90,  99]], dtype=int64)

In [22]:
# 딕셔너리 리스트를 데이터프레임으로
d = [{'kor':100, 'mat':95}, {'kor':90., 'mat':99}]
df = pd.DataFrame(data=d)
df

,kor,mat
0,100.0,95
1,90.0,99


In [23]:
df.dtypes #kor이 실수(float)로 입력됨. {'kor':90., 'mat':99}에서 90에 .을 찍었으므로

kor    float64
mat      int64
dtype: object

In [24]:
df.info() # ★

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   kor     2 non-null      float64
 1   mat     2 non-null      int64  
dtypes: float64(1), int64(1)
memory usage: 160.0 bytes


In [37]:
d = [{'kor':100, 'mat':100}, {'kor':90, 'mat':91}, {'kor':93, 'eng':100}]
df = pd.DataFrame(data=d) 
# 결측치(NaN)는 실수형(float 64)으로 나타나며 정수형 변환이 불가능함(결측치대체를 한 후 정수형변환이 가능함.)
df

,kor,mat,eng
0,100,100.0,NaN
1,90,91.0,NaN
2,93,NaN,100.0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   kor     3 non-null      int64  
 1   mat     2 non-null      float64
 2   eng     1 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 200.0 bytes


## 2.2 리스트를 이용한 데이터프레임 만들기

In [39]:
title = ['책1','책2', '책3']
price = [15000, 18000, 10000]
df = pd.DataFrame(data={'title':title, 'price':price})
# 딕셔너리 형태로 데이터를 구성하고, 이를 이용해 데이터프레임을 생성
df

,title,price
0,책1,15000
1,책2,18000
2,책3,10000


In [40]:
import numpy as np
np.c_[title, price]

array([['책1', '15000'],
       ['책2', '18000'],
       ['책3', '10000']], dtype='<U11')

In [48]:
df = pd.DataFrame(np.c_[title, price], columns=['책이름','가격'])
df

,책이름,가격
0,책1,15000
1,책2,18000
2,책3,10000


In [45]:
l = [['책1', 15000],
     ['책2', 18000],
     ['책3', 10000]]
df = pd.DataFrame(l, columns=['책이름','가격'])
df

,책이름,가격
0,책1,15000
1,책2,18000
2,책3,10000


[소상공인시장진흥공단_상가(상권)정보] https://www.data.go.kr/data/15083033/fileData.do
- 에서 데이터 다운로드, 데이터가 엄청나게 많다. 이런 데이터는 txt로 작업하거나 엑셀로 작업이 매우 비효율적이다.(용량 매우 큼)

## 2.3 read_csv

In [50]:
%ls c:\AI_x\01_download\shareData\상가정보_20250331 #폴더 내용물 확인

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 c:\AI_x\01_download\shareData\상가정보_20250331 디렉터리

2025-04-28  오후 03:51    <DIR>          .
2025-04-28  오후 03:51    <DIR>          ..
2025-04-27  오후 11:07             1,011 [필독]파일열람방법.txt
2025-04-27  오후 11:07        61,227,944 소상공인시장진흥공단_상가(상권)정보_강원_202503.csv
2025-04-27  오후 11:07       356,572,082 소상공인시장진흥공단_상가(상권)정보_경기_202503.csv
2025-04-27  오후 11:07        93,472,559 소상공인시장진흥공단_상가(상권)정보_경남_202503.csv
2025-04-27  오후 11:07        76,425,275 소상공인시장진흥공단_상가(상권)정보_경북_202503.csv
2025-04-27  오후 11:07        40,070,868 소상공인시장진흥공단_상가(상권)정보_광주_202503.csv
2025-04-27  오후 11:07        61,715,005 소상공인시장진흥공단_상가(상권)정보_대구_202503.csv
2025-04-27  오후 11:07        41,491,092 소상공인시장진흥공단_상가(상권)정보_대전_202503.csv
2025-04-27  오후 11:07        84,550,600 소상공인시장진흥공단_상가(상권)정보_부산_202503.csv
2025-04-27  오후 11:07       294,549,483 소상공인시장진흥공단_상가(상권)정보_서울_202503.csv
2025-04-27  오후 11:07         9,303,432 소상공인시장진흥공단_상가(상권)정보_세종_202503.csv
2025-04-27  오후 11:07        28,457,4

In [52]:
df = pd.read_csv(r'C:/AI_x/01_download/shareData/상가정보_20250331/소상공인시장진흥공단_상가(상권)정보_세종_202503.csv',
                sep=',',
                low_memory=False, #데이터 용량이 클 경우 기입
                nrows=30 # 처음 30행만 가져오라는 명령어
                )
df.shape

(30, 39)

In [53]:
df #그냥 무지성으로 열면 이모양남.

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220700019175,세종수이비인후과의원,NaN,Q1,보건의료,Q102,의원,Q10201,내과/소아과 의원,Q86201,...,3611011600200050002000001,행정중심복합도시,세종특별자치시 세종특별자치시 해밀3로 43,NaN,30092,NaN,NaN,NaN,127.264487,36.528585
1,MA010120220800005576,우보설렁탕,NaN,I2,음식,I201,한식,I20102,국/탕/찌개류,I56111,...,4473025023100230051035586,NaN,세종특별자치시 세종특별자치시 조치원읍 문화로 24,339885.0,30025,NaN,NaN,NaN,127.301068,36.603770
2,MA010120220800006182,메이빈,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,3611010300106640011000001,강남프라자,세종특별자치시 세종특별자치시 시청대로 127,339008.0,30151,NaN,3.0,NaN,127.286081,36.479674
3,MA010120220800008970,명품동태탕,NaN,I2,음식,I201,한식,I20102,국/탕/찌개류,I56111,...,3611035024103040008000001,행복마을,세종특별자치시 세종특별자치시 장군면 월현윗길 29,339914.0,30055,NaN,1.0,NaN,127.223073,36.501346
4,MA010120220800024017,컴119,NaN,S2,수리·개인,S201,컴퓨터 수리,S20101,컴퓨터/노트북/프린터 수리업,S95110,...,3611010600102180000000001,골든타원,세종특별자치시 세종특별자치시 노을3로 99,339014.0,30130,NaN,2.0,NaN,127.253843,36.479828
5,MA010120220800047182,포지션,NaN,G2,소매,G216,안경·정밀기기 소매,G21603,사진기/기타 광학기기 소매업,G47829,...,3611034022102850004000001,NaN,세종특별자치시 세종특별자치시 금남면 발산1길 67,339833.0,30083,NaN,NaN,NaN,127.283364,36.456746
6,MA010120220800046344,이화공인중개사사무소,NaN,L1,부동산,L102,부동산 서비스,L10203,부동산 중개/대리업,L68221,...,3611010700106840001000001,나릿재마을2단지,세종특별자치시 세종특별자치시 어울누리로 67,339003.0,30128,NaN,NaN,NaN,127.265373,36.487283
7,MA010120220800046670,김가네칼국수,NaN,I2,음식,I201,한식,I20105,국수/칼국수,I56112,...,4473025029102660002000001,NaN,세종특별자치시 세종특별자치시 조치원읍 큰말2길 3,339807.0,30030,NaN,NaN,NaN,127.289301,36.595365
8,MA010120220800048962,굿모닝보청기,NaN,G2,소매,G215,의약·화장품 소매,G21502,의료기기 소매업,G47812,...,4473025021100010014037459,NaN,세종특별자치시 세종특별자치시 조치원읍 조치원7길 6,339883.0,30024,NaN,1.0,NaN,127.300613,36.601726
9,MA010120220800042909,청년피자,조치원점,I2,음식,I210,기타 간이,I21003,피자,I56192,...,3611025028102670008000001,주건축물제,세종특별자치시 세종특별자치시 조치원읍 행복10길 9,339887.0,30021,NaN,2.0,NaN,127.291133,36.604147


In [58]:
# 판다스 디스플레이 옵션
pd.options.display.max_columns = 39 #기본값은 20으로 되어있음. 39열까지 display 가능
#max_columns	화면에 표시할 최대 열 수
pd.options.display.max_rows
#max_rows	화면에 표시할 최대 행 수(기본값 60)
df.shape

(30, 39)

In [59]:
df.head(2)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220700019175,세종수이비인후과의원,NaN,Q1,보건의료,Q102,의원,Q10201,내과/소아과 의원,Q86201,일반의원,36,세종특별자치시,36110,세종특별자치시,36110525,해밀동,3611011600,해밀동,3611011600200050002,2,산,5,2.0,세종특별자치시 세종특별자치시 해밀동 산5-2,361103352828,세종특별자치시 세종특별자치시 해밀3로,43,NaN,3611011600200050002000001,행정중심복합도시,세종특별자치시 세종특별자치시 해밀3로 43,NaN,30092,NaN,NaN,NaN,127.264487,36.528585
1,MA010120220800005576,우보설렁탕,NaN,I2,음식,I201,한식,I20102,국/탕/찌개류,I56111,한식 일반 음식점업,36,세종특별자치시,36110,세종특별자치시,36110250,조치원읍,3611025023,조치원읍,3611025023100230051,1,대지,23,51.0,세종특별자치시 세종특별자치시 조치원읍 평리 23-51,361103258043,세종특별자치시 세종특별자치시 조치원읍 문화로,24,NaN,4473025023100230051035586,NaN,세종특별자치시 세종특별자치시 조치원읍 문화로 24,339885.0,30025,NaN,NaN,NaN,127.301068,36.603770


In [60]:
df.head(1).T

,0
상가업소번호,MA010120220700019175
상호명,세종수이비인후과의원
지점명,NaN
상권업종대분류코드,Q1
상권업종대분류명,보건의료
상권업종중분류코드,Q102
상권업종중분류명,의원
상권업종소분류코드,Q10201
상권업종소분류명,내과/소아과 의원
표준산업분류코드,Q86201


In [61]:
# (1) CSV 파일 불러오기(기본값)
# encording=utf-8, sep=',', csv파일의 첫번째 줄을 header로 인식, #이 있는 줄도 데이터로 인식
member = pd.read_csv('data/ch11_member.csv')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30
5,#홍길뜽,24,non@not.com,없는구,2000-01-01


In [62]:
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     6 non-null      object
 1   Age      6 non-null      int64 
 2   Email    6 non-null      object
 3   Address  6 non-null      object
 4   Birth    6 non-null      object
dtypes: int64(1), object(4)
memory usage: 368.0+ bytes


In [67]:
# 형변환 Age열:int64 -> int16
member['Age'] = member['Age'].astype(np.int16) #'int' = np.int32와 동일함.
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     6 non-null      object
 1   Age      6 non-null      int16 
 2   Email    6 non-null      object
 3   Address  6 non-null      object
 4   Birth    6 non-null      object
dtypes: int16(1), object(4)
memory usage: 332.0+ bytes


In [68]:
# 형변환 Birth열 : object(문자) -> datetime64
member['Birth'].astype('datetime64')

0   1999-01-01
1   1999-12-01
2   1999-11-01
3   1999-05-01
4   1999-01-30
5   2000-01-01
Name: Birth, dtype: datetime64[ns]

In [69]:
member['Birth_as'] = member['Birth'].astype('datetime64')
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Name      6 non-null      object        
 1   Age       6 non-null      int16         
 2   Email     6 non-null      object        
 3   Address   6 non-null      object        
 4   Birth     6 non-null      object        
 5   Birth_as  6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int16(1), object(4)
memory usage: 380.0+ bytes


In [70]:
member['Birth_to'] = pd.to_datetime(member['Birth']) # astype()함수보다 안전한 형변환
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Name      6 non-null      object        
 1   Age       6 non-null      int16         
 2   Email     6 non-null      object        
 3   Address   6 non-null      object        
 4   Birth     6 non-null      object        
 5   Birth_as  6 non-null      datetime64[ns]
 6   Birth_to  6 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int16(1), object(4)
memory usage: 428.0+ bytes


In [72]:
# (2) 특정행을 제외하고 csv파일읽기
member = pd.read_csv('data/ch11_membercp949.csv',
                    encoding='cp949',
                    skiprows=[1, 6]) # 1, 6번째 행은 제외하고 읽어오기
member

,Name,Age,Email,Address,Birth
0,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
1,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
2,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
3,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [77]:
# (3) 주석(#)제외, datetime형 필드 지정하여 읽기
member = pd.read_csv('data/ch11_membertab.csv',
                    sep='\t',
                    comment='#',
                    parse_dates=['Birth']) # datetime형으로 읽어올 필드 지정
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [79]:
# (4) sep='/', 상위5행만 읽어오기
member = pd.read_csv('data/ch11_membersep.csv',
                    sep='|',
                    nrows=5)
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


## 2.4 패키지에서 가져오기 : iris 데이터
- 3가지 방법이 있음.
    - iris 가져오기 방법 1 : sklearn패키지(머신러닝 패키지)에서 가져오기
    - iris 가져오기 방법 2 : statsmodels (R 데이터)
    - iris 가져오기 방법 3 : seabron(시각화 패키지)

### iris 가져오기 방법 1 : sklearn패키지(머신러닝 패키지)에서 가져오기

In [85]:
from sklearn import datasets
# sklearn : 머신러닝을 위한 함수(전처리 함수, 머신러닝 함수, 성능평가를 위한 함수, 학습데이터셋)
iris = datasets.load_iris()
iris.keys() # 딕셔너리 형태

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [86]:
print(iris.DESCR) #iris 데이터 셋에 대한 설명

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [87]:
iris.data # 독립변수
iris['data'][:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [88]:
iris.feature_names #독립변수의 열이름
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [94]:
# 데이터 프레임에서 열이름
columns = [col[:-5].replace(' ','_') for col in iris.feature_names]+['species']
columns

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

In [97]:
iris.target
iris['target'] # 머신러닝시 종속변수

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [99]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [101]:
iris['target_names'][iris['target']]
iris.target_names[iris.target]

array(['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolo

In [103]:
# 독립변수
data = iris.data
# 종속변수
target = iris.target_names[iris.target]
data.shape, target.shape # 독립변수와 종속변수의 차원

((150, 4), (150,))

In [104]:
# 독립변수와 종속변수를 stack한 후 데이터프레임으로 
pd.DataFrame(data = np.hstack( (data, target.reshape(-1,1) ) ),
            columns=columns)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
